In [1]:
!pip install -q git+git://github.com/deepmind/optax.git
!pip install -q git+https://github.com/deepmind/dm-haiku

     |████████████████████████████████| 51kB 6.3MB/s 


In [2]:
from typing import Any, Generator, Mapping, Tuple

from absl import app
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
import optax
import tensorflow as tf
import tensorflow_datasets as tfds
from optax._src import transform
from jax import jit, grad, vmap
from jax.tree_util import tree_structure
from numpy.random import default_rng

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils


import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

np.set_printoptions(precision=2,suppress=True)
np.random.seed(0)


In [4]:
# for every environment, pick at random a strongest predictor (does not have to be unique) + random small coefficients for all others
# EXCEPT for a subset of predictors which are consistent across all environments

def generate_synthetic_dataset(E,ntrain,glob_sigma,sig_sigma,K,rand_strong_coefs,inv_weak_coefs,true_pred_indices,n_informative,n_redundant,use_sklearn):
    # more info for sklearn dataset generation: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html#sklearn.datasets.make_classification
    dist = torch.distributions.Uniform(-1, 1)
    if use_sklearn:
      X,y = make_classification(n_samples=E, 
                          n_features=K, 
                          n_informative=n_informative,
                          n_redundant=n_redundant, 
                          n_repeated=0, 
                          n_classes=2, 
                          n_clusters_per_class=2, 
                          weights=None, 
                          flip_y=sig_sigma, 
                          class_sep=glob_sigma, 
                          hypercube=True, 
                          shift=0.0, 
                          scale=1.0, 
                          shuffle=False, 
                          random_state=np.random.seed(0))
    else:
      rng = default_rng()
      #np.random.seed(100)
      X = np.random.normal(0, glob_sigma, (E, K))
      #y = np.random.randint(2,size=E)
      y = rng.choice(2, size=E, replace=True)

      for e in range(E):
        # new seed
        #np.random.seed(e)
        # pick a subset of best predictors at random 
        #rand_best_indices = np.random.randint(K, size=3)
        rand_best_indices = rng.choice(K, size=1, replace=False)

        # check to make sure these randoms aren't out true pred
        condition = True
        while condition:
          for i,rbp in enumerate(rand_best_indices):
            if rbp in true_pred_indices:
              #rand_best_indices[i] = np.random.randint(K, size=1)
              rand_best_indices[i] = rng.choice(K, size=1, replace=False)
              condition = True
              break
            else:
              condition = False
        #print(true_pred_indices,rand_best_indices)
        # set the coefficients at the right row 
        # that separates each class
        if y[e] == 1:
          X[e,true_pred_indices] = inv_weak_coefs # +  np.random.normal(0,sig_sigma)
          X[e,rand_best_indices] = rand_strong_coefs + dist.sample(X[e,rand_best_indices].shape)  #np.random.normal(0,sig_sigma)

        elif y[e] == 0:
          X[e,true_pred_indices] = 0 #+  np.random.normal(0,sig_sigma) #-inv_weak_coefs 
          X[e,rand_best_indices] = 0 + dist.sample(X[e,rand_best_indices].shape)


    #Z = np.c_[ X, y ]   
    #print(np.corrcoef(Z.T))
    print(X,y)
    return y,X



In [5]:
class ANDMaskState(optax.OptState):
  """Stateless.""" # Following optax code style

def and_mask(agreement_threshold: float) -> optax.GradientTransformation:
  def init_fn(_):
    # Required by optax
    return ANDMaskState()

  def update_fn(updates, opt_state, params=None):

    def and_mask(update):
      # Compute the masked gradients for a single parameter tensor
      mask = jnp.abs(jnp.mean(jnp.sign(update), 0)) >= agreement_threshold
      mask = mask.astype(jnp.float32)
      avg_update = jnp.mean(update, 0)
      mask_t = mask.sum() / mask.size
      update = mask * avg_update * (1. / (1e-10 + mask_t))
      return update

    del params # Following optax code style
    
    # Compute the masked gradients over all parameters

    # jax.tree_map maps a function (lambda function in this case) over a pytree to produce a new pytree.
    updates = jax.tree_map(lambda x: and_mask(x), updates)
    return updates, opt_state

  return transform.GradientTransformation(init_fn, update_fn)

In [27]:
def sparse_logistic_regression_synthetic_data(train, test, adam_lr=1e-3, agreement_threshold=0.0,
                               use_ilc=False, l1_coef=1e-4, l2_coef=1e-4,
                               epochs=10001, Verbose=False, training=True, n_classes=2, normalizer=255.):

  
    OptState = Any
    Batch = Mapping[str, np.ndarray]

    def evaluate(params, validation_data, validation_labels):
    
        logits = net.apply(params, validation_data)
        preds = jax.nn.log_softmax(logits)
        pred_labels = np.argmax(preds, axis=1)

        acc = accuracy(params, validation_data, validation_labels)
        # As mentioned before, data is unbalanced, hence, the accuracy itself is not 
        # enough for evaluating the performance of the model.
        # print(outputs,local_labels.cpu().detach().numpy())
        cm = confusion_matrix(pred_labels.transpose(), validation_labels.transpose())
        #sns.set_theme()
        #plt.figure()
        #ax = sns.heatmap(cm)
        print('\nConfusion Matrix: ', cm)
        precision,recall,fscore,_ = precision_recall_fscore_support(validation_labels, pred_labels)
        print('\nAccuracy: ', acc,'\nPrecision: ',precision,'\nRecall: ', recall,'\nF-score: ', fscore)


    training_accs = []
    testing_accs = []

    def net_fn(batch) -> jnp.ndarray:
    
        x = jnp.array(batch, jnp.float32) / normalizer
        mlp = hk.Sequential([
            hk.Flatten(),
            hk.Linear(n_classes),
        ])
        return mlp(x)


    # Make the network and optimiser.
    net = hk.without_apply_rng(hk.transform(net_fn))

    
       
    # Training loss (cross-entropy).
    def loss(params: hk.Params, batch, label) -> jnp.ndarray:
        """Compute the loss of the network, including L2."""
        logits = net.apply(params, batch)
        labels = jax.nn.one_hot(label, n_classes)

        # Cross Entropy Loss
        softmax_xent = -jnp.sum(labels * jax.nn.log_softmax(logits))
        softmax_xent /= labels.shape[0]
        return softmax_xent
        
    # Regularization loss (L1,L2).
    def regularization_loss(params: hk.Params) -> jnp.ndarray:
        """Compute the regularization loss of the network, applied after ILC."""

        # L1 Loss
        sum_in_layer = lambda p: jnp.sum(jnp.abs(p))
        sum_p_layers = [sum_in_layer(p) for p in jax.tree_leaves(params)]
        l1_loss = sum(sum_p_layers)

        # L2 Loss
        l2_loss = 0.5 * sum(jnp.sum(jnp.square(p)) for p in jax.tree_leaves(params))

        return l2_coef * l2_loss + l1_coef * l1_loss


    @jax.jit
    def gradient_per_sample(params, batch, label):
        batch, label = jnp.expand_dims(batch,1), jnp.expand_dims(label,1)
        return vmap(grad(loss), in_axes=(None, 0, 0))(params, batch, label)

    
    gradient = jax.jit(grad(loss))
    gradient_reg = jax.jit(grad(regularization_loss))

    # Evaluation metric (classification accuracy).
    @jax.jit
    def accuracy(params: hk.Params, batch, label) -> jnp.ndarray:
        predictions = net.apply(params, batch)
        return jnp.mean(jnp.argmax(predictions, axis=-1) == label)


    
    @jax.jit
    def update(
        params: hk.Params,
        opt_state: OptState,
        batch, label, agreement
        ) -> Tuple[hk.Params, OptState]:
        """Learning rule (stochastic gradient descent)."""
        # grads = jax.grad(loss)(params, batch, label)
        # grads_masked = (gradient_per_sample if use_ilc else gradient)(params, batch, label) # (gradient_per_sample)(params, batch, label)
        # sum_grad_masked_regularized = jax.tree_multimap(lambda x,y:x+y,grads_masked,gradient_reg(params))
        # grads = sum_grad_masked_regularized
        # updates, opt_state = opt.update(grads, opt_state)
        # new_params = optax.apply_updates(params, updates)

        grads_samples = gradient_per_sample(params, batch, label)
        ANDmask = and_mask(agreement)

        masked_grads,_ = ANDmask.update(grads_samples, opt_state)
        reg_grads = gradient_reg(params)

        sum_grad_masked_regularized = jax.tree_multimap(lambda x,y:x+y,masked_grads,reg_grads)
 
        updates,_ = opt.update(sum_grad_masked_regularized, opt_state)

        new_params = optax.apply_updates(params, updates)

        return new_params, opt_state

    # We maintain avg_params, the exponential moving average of the "live" params.
    # avg_params is used only for evaluation.
    # For more, see: https://doi.org/10.1137/0330046
    @jax.jit
    def ema_update(
        avg_params: hk.Params,
        new_params: hk.Params,
        epsilon: float = 0.001,
    ) -> hk.Params:
        return jax.tree_multimap(lambda p1, p2: (1 - epsilon) * p1 + epsilon * p2,
                                avg_params, new_params)

    if training is False:
        return net, accuracy
    else:
        if(use_ilc):

            use_ilc = False

            # opt = optax.chain(and_mask(agreement_threshold) if use_ilc else optax.identity(),optax.adam(adam_lr))

            # schedule_fn = optax.exponential_decay(adam_lr, # Note the minus sign!
            # 1,
            # 0.9)
            # opt = optax.chain(optax.sgd(adam_lr), optax.scale_by_schedule(schedule_fn)) # Or Adam could be used
            opt = optax.chain(optax.sgd(adam_lr)) # Or Adam could be used

            # Initialize network and optimiser; note we draw an input to get shapes.
            params = avg_params = net.init(jax.random.PRNGKey(42), next(train)[0])
            opt_state = opt.init(params)

            # Train/eval loop. WITHOUT ILC
            print("Begin training with ILC")
            for step in range(np.int(.5*epochs)):
                if step % np.int(epochs/10) == 0:
                    # Periodically evaluate classification accuracy on train & test sets.
                    Batch = next(train)
                    train_accuracy = accuracy(avg_params, Batch[0], Batch[1])
                    train_accuracy = jax.device_get(train_accuracy)
                    Batch = next(test)
                    test_accuracy = accuracy(avg_params, Batch[0], Batch[1])
                    test_accuracy = jax.device_get(test_accuracy)
                    training_accs.append(train_accuracy)
                    testing_accs.append(test_accuracy)
                    if Verbose:
                        print(f"[ILC Off, Step {step}] Train accuracy/Test accuracy: "
                                f"{train_accuracy:.3f} / {test_accuracy:.3f}.")

                # Do SGD on a batch of training examples.
                Batch = next(train)
                params, opt_state = update(params, opt_state, Batch[0], Batch[1], 0.)
                avg_params = ema_update(avg_params, params)


            use_ilc = True

            # opt = optax.chain(optax.adam(adam_lr))

            # Initialize network and optimiser; note we draw an input to get shapes.
            opt_state = opt.init(params)
            
            # Train/eval loop. WITH ILC
            for step in range(np.int(.5*epochs)):
                if step % np.int(epochs/10) == 0:
                    # Periodically evaluate classification accuracy on train & test sets.
                    Batch = next(train)
                    train_accuracy = accuracy(avg_params, Batch[0], Batch[1])
                    train_accuracy = jax.device_get(train_accuracy)
                    Batch = next(test)
                    test_accuracy = accuracy(avg_params, Batch[0], Batch[1])
                    test_accuracy = jax.device_get(test_accuracy)
                    training_accs.append(train_accuracy)
                    testing_accs.append(test_accuracy)
                    if Verbose:
                        print(f"[ILC On, Step {step}] Train accuracy/Test accuracy: "
                                f"{train_accuracy:.3f} / {test_accuracy:.3f}.")

                # Do SGD on a batch of training examples.
                Batch = next(train)
                params, opt_state = update(params, opt_state, Batch[0], Batch[1], agreement_threshold)
                avg_params = ema_update(avg_params, params)

            print("Evaluating with ILC:")
            Batch = next(test)
            test_data, test_labels = Batch[0], Batch[1]
            evaluate(avg_params, test_data, test_labels)


            return params, training_accs, testing_accs

        else:

             # schedule_fn = optax.exponential_decay(adam_lr, # Note the minus sign!
            # 1,
            # 0.9)
            # opt = optax.chain(optax.sgd(adam_lr), optax.scale_by_schedule(schedule_fn)) # Or Adam could be used
            opt = optax.chain(optax.sgd(adam_lr))

            # Initialize network and optimiser; note we draw an input to get shapes.
            params = avg_params = net.init(jax.random.PRNGKey(42), next(train)[0])
            opt_state = opt.init(params)

            use_ilc=False
            # Train/eval loop. 
            print("Begin training without ILC")
            for step in range(np.int(epochs)):
                if step % np.int(epochs/10) == 0:
                    # Periodically evaluate classification accuracy on train & test sets.
                    Batch = next(train)
                    train_accuracy = accuracy(avg_params, Batch[0], Batch[1])
                    train_accuracy = jax.device_get(train_accuracy)
                    Batch = next(test)
                    test_accuracy = accuracy(avg_params, Batch[0], Batch[1])
                    test_accuracy = jax.device_get(test_accuracy)
                    training_accs.append(train_accuracy)
                    testing_accs.append(test_accuracy)
                    if Verbose:
                        print(f"[ADAM, Step {step}] Train accuracy/Test accuracy: "
                                f"{train_accuracy:.3f} / {test_accuracy:.3f}.")
                        
                # Do SGD on a batch of training examples.
                Batch = next(train)
                params, opt_state = update(params, opt_state, Batch[0], Batch[1], 0.)
                avg_params = ema_update(avg_params, params)
            
            print("Evaluating without ILC:")
            Batch = next(test)
            test_data, test_labels = Batch[0], Batch[1]
            evaluate(avg_params, test_data, test_labels)

            
            return params, training_accs, testing_accs


In [60]:
rng = default_rng()
E = 5000 # number of environments
train_batch_size = np.float32(.5*E)
sigma = [6] # amount of noise 
signal_noise = [4]
K = 45 # number of predictors (at least 40)
n_informative = 20
n_redundant = 0
nepochs = 1000
weights=[0.7,0.3]
rand_strong_coefs = 5.
inv_weak_coefs = 0.1
use_sklearn = False
# true invariant predictor indices 
#true_pred_indices = np.random.randint(K, size=n_informative)
true_pred_indices = rng.choice(K, size=n_informative, replace=False)
true_pred_indices
agreement = 0.4
l1_sgd = 0.
l2_sgd = 0.
l1_ilc = 1e-4
l2_ilc = 1e-4

In [61]:
from sklearn.datasets import make_classification

for global_noise in sigma:
    for sig_noise in signal_noise:
        print(f"Global noise: {global_noise}, Signal noise: {sig_noise}")
        y,x = generate_synthetic_dataset(E,train_batch_size,global_noise,sig_noise,K,rand_strong_coefs,inv_weak_coefs,true_pred_indices,n_informative,n_redundant,use_sklearn)
        
        test_batch_size = E-train_batch_size
        # ds = tfds.load("mnist:3.*.*", split=splits[m]).cache().repeat()
        # print(ds)
        all_dataset = tf.data.Dataset.from_tensor_slices((x, y)).repeat()
        train_dataset = all_dataset.take(train_batch_size).repeat()
        test_dataset = all_dataset.skip(train_batch_size).repeat()
        # print(train_dataset)
        # print(test_dataset)
        train_dataset = train_dataset.shuffle(10.0 * train_batch_size, seed=0)
        train_dataset = train_dataset.batch(train_batch_size)
        train_dataset = iter(tfds.as_numpy(train_dataset))

        test_dataset = test_dataset.shuffle(10.0 * test_batch_size, seed=0)
        test_dataset = test_dataset.batch(test_batch_size)
        test_dataset = iter(tfds.as_numpy(test_dataset))
        
        print("***Using Elastic Net***")
        # sgd
        params = sparse_logistic_regression_synthetic_data(train_dataset, test_dataset, adam_lr=1e-1, agreement_threshold=0.,
                                       use_ilc=False, l1_coef=l1_sgd, l2_coef=l2_sgd,
                                       epochs=nepochs, Verbose=True, training=True, n_classes=2, normalizer=1.)
        # print(params)
        # sgd-ilc
        params = sparse_logistic_regression_synthetic_data(train_dataset, test_dataset, adam_lr=1e-1, agreement_threshold=agreement,
                                use_ilc=True, l1_coef=l1_ilc, l2_coef=l2_ilc,
                                epochs=nepochs, Verbose=True, training=True, n_classes=2, normalizer=1.)
        # print(params)

Global noise: 6, Signal noise: 4
[[  0.1    4.54  -9.24 ...   0.1    0.1   -8.84]
 [  0.1    7.8    4.16 ...   0.1    0.1   -8.57]
 [  0.1   16.04  -2.45 ...   0.1    0.1   -7.4 ]
 ...
 [  0.1    9.62  -6.1  ...   0.1    0.1   -4.78]
 [  0.1  -12.28   1.05 ...   0.1    0.1    9.16]
 [  0.   -17.01   2.01 ...   0.     0.     0.19]] [1 1 1 ... 1 1 0]
***Using Elastic Net***
Begin training without ILC
[ADAM, Step 0] Train accuracy/Test accuracy: 0.480 / 0.494.
[ADAM, Step 100] Train accuracy/Test accuracy: 0.478 / 0.492.
[ADAM, Step 200] Train accuracy/Test accuracy: 0.485 / 0.506.
[ADAM, Step 300] Train accuracy/Test accuracy: 0.475 / 0.495.
[ADAM, Step 400] Train accuracy/Test accuracy: 0.502 / 0.507.
[ADAM, Step 500] Train accuracy/Test accuracy: 0.513 / 0.549.
[ADAM, Step 600] Train accuracy/Test accuracy: 0.537 / 0.562.
[ADAM, Step 700] Train accuracy/Test accuracy: 0.569 / 0.580.
[ADAM, Step 800] Train accuracy/Test accuracy: 0.571 / 0.590.
[ADAM, Step 900] Train accuracy/Test accur